In [9]:
import optuna
import xgboost as xgb
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# --- 1. Preparazione Dati ---
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

X_train_safe = np.asarray(X_train)
y_train_safe = np.asarray(y_train)
X_test_safe  = np.asarray(X_test)
y_test_safe  = np.asarray(y_test)

# wrap training and test sets in DMatrix (GPU)
dtrain = xgb.DMatrix(X_train_safe, label=y_train_safe) 
dtest = xgb.DMatrix(X_test_safe, label=y_test_safe)


# --- 2. Objective Function ---
def objective(trial):
    params = {
        "objective": "binary:logistic",
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        #"tree_method": "hist",
        #"device": "cuda",
        "seed": 42,
        "eval_metric": "error",
    }

    num_round = trial.suggest_int("n_estimators", 100, 1000)

    xgb_cv = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_round,
        early_stopping_rounds=20,
        nfold=5,
        stratified=True,
        seed=42,
        metrics="error",
    )

    return 1.0 - xgb_cv["test-error-mean"].min()

# --- 3. Run Optuna study ---
print("--- Bayesian Optimization ---")
study = optuna.create_study(direction="maximize", study_name="XGBoost_BreastCancer_GPU")
study.optimize(objective, n_trials=50, show_progress_bar=False)

# --- 4. Results ---
print("\n--- Optimization's results ---")
print(f"Miglior Trial (Tentativo #{study.best_trial.number})")
print(f"Accuratezza Migliore (CV): {study.best_value:.4f}")
print("Migliori Iperparametri:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# --- 5. Validazione Finale sul Test Set ---
best_params = study.best_params

num_boost_round = best_params.pop('n_estimators')

best_params.update({
    "objective": "binary:logistic",
    "tree_method": "hist",
    "device": "cuda",
    "seed": 42,
    "eval_metric": "error",
})

final_model = xgb.train(best_params, dtrain, num_boost_round=num_boost_round)
y_pred = final_model.predict(dtest)
y_pred_labels = (y_pred > 0.5).astype(int)

final_acc = accuracy_score(y_test_safe, y_pred_labels)
print(f"\nFinal accuracy on test set: {final_acc:.4f}")

[I 2025-12-05 12:54:45,059] A new study created in memory with name: XGBoost_BreastCancer_GPU


--- Bayesian Optimization ---


[I 2025-12-05 12:54:45,242] Trial 0 finished with value: 0.9604395604395605 and parameters: {'learning_rate': 0.029562560414104135, 'max_depth': 9, 'subsample': 0.5487264864204608, 'colsample_bytree': 0.5481894960925886, 'reg_alpha': 4.593571900763969e-05, 'reg_lambda': 4.5993704239514684e-07, 'n_estimators': 750}. Best is trial 0 with value: 0.9604395604395605.
[I 2025-12-05 12:54:45,413] Trial 1 finished with value: 0.9758241758241758 and parameters: {'learning_rate': 0.12814103508685787, 'max_depth': 4, 'subsample': 0.5267669944769173, 'colsample_bytree': 0.5562493784774314, 'reg_alpha': 1.7843445205772853e-08, 'reg_lambda': 4.5802941681997936e-08, 'n_estimators': 768}. Best is trial 1 with value: 0.9758241758241758.
[I 2025-12-05 12:54:45,680] Trial 2 finished with value: 0.9582417582417583 and parameters: {'learning_rate': 0.03895676992783409, 'max_depth': 7, 'subsample': 0.8507707481485676, 'colsample_bytree': 0.8587758342938387, 'reg_alpha': 0.0021453387342647854, 'reg_lambda': 


--- Optimization's results ---
Miglior Trial (Tentativo #20)
Accuratezza Migliore (CV): 0.9780
Migliori Iperparametri:
  learning_rate: 0.11541734345878538
  max_depth: 6
  subsample: 0.5842248240838432
  colsample_bytree: 0.5649538414850059
  reg_alpha: 0.00023733792461235416
  reg_lambda: 0.1258348593049783
  n_estimators: 676

Final accuracy on test set: 0.9737


In [10]:
# --- 6. Visualizzazione Avanzata ---
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice

# A. Storia dell'ottimizzazione
# Vediamo se il modello sta migliorando nel tempo
fig1 = plot_optimization_history(study)
fig1.show()

# B. Importanza degli Iperparametri
# Quale parametro impatta di più l'accuratezza? (Spesso è il Learning Rate)
fig2 = plot_param_importances(study)
fig2.show()

# C. Slice Plot
# Vediamo dove si concentrano i punti migliori per ogni parametro
fig3 = plot_slice(study)
fig3.show()